In [19]:
import json
import pandas as pd
from sqlanalyzer import column_parser

In [52]:
with open('./sqlanalyzer/data.json', 'r') as f:
    query_dict = json.load(f)

In [53]:
print(json.dumps(query_dict, indent=2), '\n\n\n')

{
  "no alias": {
    "a": "SELECT DISTINCT anonymous_id,                    user_id    FROM mapbox_customer_data.segment_identifies    WHERE dt >= '2018-07-01'      AND anonymous_id IS NOT NULL AND user_id IS NOT NULL ",
    "b": "SELECT id,           email,           created    FROM mapbox_customer_data.accounts WHERE cast(dt AS DATE) = CURRENT_DATE - INTERVAL '1' DAY ",
    "c": "SELECT anonymous_id AS anon_id_ad,           context_campaign_name,           min(TIMESTAMP) AS min_exposure    FROM mapbox_customer_data.segment_pages    WHERE dt >= '2018-07-01'      AND context_campaign_name IS NOT NULL    GROUP BY 1, 2",
    "d": "SELECT DISTINCT anonymous_id AS anon_id_event,                    original_timestamp,                    event,                    context_traits_email    FROM mapbox_customer_data.segment_tracks    WHERE dt >= '2018-07-01'      AND event LIKE 'submitted_%form' AND context_traits_email IS NOT NULL ",
    "e": {
      "sfdc_accounts": "sfdc.accounts",
      "sf

## looking at each subquery

In [54]:
query = query_dict['no alias']['a']

In [39]:
db_fields_1 = pd.DataFrame({'db_table': 'mapbox_customer_data.segment_identifies', 
            'all_columns': ['anonymous_id', 'user_id', 'service', 'service_metadata', 'service_metadata_version', 'account', 'num_requests', 'dt']})
db_fields_1


,db_table,all_columns
0,mapbox_customer_data.segment_identifies,anonymous_id
1,mapbox_customer_data.segment_identifies,user_id
2,mapbox_customer_data.segment_identifies,service
3,mapbox_customer_data.segment_identifies,service_metadata
4,mapbox_customer_data.segment_identifies,service_metadata_version
5,mapbox_customer_data.segment_identifies,account
6,mapbox_customer_data.segment_identifies,num_requests
7,mapbox_customer_data.segment_identifies,dt


In [40]:
db_fields_2 = pd.DataFrame({'db_table': 'mapbox_customer_data.accounts', 
            'all_columns': ['id', 'user_id', 'email', 'created', 'service_metadata_version', 'account', 'num_requests', 'dt']})
db_fields_2

,db_table,all_columns
0,mapbox_customer_data.accounts,id
1,mapbox_customer_data.accounts,user_id
2,mapbox_customer_data.accounts,email
3,mapbox_customer_data.accounts,created
4,mapbox_customer_data.accounts,service_metadata_version
5,mapbox_customer_data.accounts,account
6,mapbox_customer_data.accounts,num_requests
7,mapbox_customer_data.accounts,dt


In [48]:
db_fields_3 = pd.DataFrame({'db_table': 'mapbox_customer_data.segment_pages', 
            'all_columns': ['anonymous_id', 'context_campaign_name', 'service', 'service_metadata', 'service_metadata_version', 'account', 'num_requests', 'dt']})
db_fields_3

,db_table,all_columns
0,mapbox_customer_data.segment_pages,anonymous_id
1,mapbox_customer_data.segment_pages,context_campaign_name
2,mapbox_customer_data.segment_pages,service
3,mapbox_customer_data.segment_pages,service_metadata
4,mapbox_customer_data.segment_pages,service_metadata_version
5,mapbox_customer_data.segment_pages,account
6,mapbox_customer_data.segment_pages,num_requests
7,mapbox_customer_data.segment_pages,dt


In [42]:
db_fields_4 = pd.DataFrame({'db_table': 'mapbox_customer_data.segment_tracks', 
            'all_columns': ['anonymous_id', 'original_timestamp', 'event', 'context_traits_email', 'service_metadata_version', 'account', 'num_requests', 'dt']})
db_fields_4

,db_table,all_columns
0,mapbox_customer_data.segment_tracks,anonymous_id
1,mapbox_customer_data.segment_tracks,original_timestamp
2,mapbox_customer_data.segment_tracks,event
3,mapbox_customer_data.segment_tracks,context_traits_email
4,mapbox_customer_data.segment_tracks,service_metadata_version
5,mapbox_customer_data.segment_tracks,account
6,mapbox_customer_data.segment_tracks,num_requests
7,mapbox_customer_data.segment_tracks,dt


In [49]:
df = db_fields_1.append(db_fields_2, ignore_index=True)
df = df.append(db_fields_3, ignore_index=True)
df = df.append(db_fields_4, ignore_index=True)

In [50]:
df

,db_table,all_columns
0,mapbox_customer_data.segment_identifies,anonymous_id
1,mapbox_customer_data.segment_identifies,user_id
2,mapbox_customer_data.segment_identifies,service
3,mapbox_customer_data.segment_identifies,service_metadata
4,mapbox_customer_data.segment_identifies,service_metadata_version
5,mapbox_customer_data.segment_identifies,account
6,mapbox_customer_data.segment_identifies,num_requests
7,mapbox_customer_data.segment_identifies,dt
8,mapbox_customer_data.accounts,id
9,mapbox_customer_data.accounts,user_id


In [45]:
def extract_subquery_fields(query, db_fields):
    formatter = column_parser.Parser(query)
    formatted = formatter.format_query(query)
    table_alias_mapping = formatter.get_table_names(formatted.split('\n'))
    fields = formatter.match_queried_fields(query, db_fields)
    return fields


In [55]:
new_dict = {}
new_dict['no alias'] = query_dict['no alias']
for k,v in query_dict['no alias'].items():
    if k in ('a', 'b', 'c', 'd'):
        new_dict['no alias'][k] = extract_subquery_fields(query_dict['no alias'][k], df)

In [56]:
new_dict

{'no alias': {'a': [{'database_name': 'mapbox_customer_data',
    'table_name': 'segment_identifies',
    'column_name': 'dt'},
   {'database_name': 'mapbox_customer_data',
    'table_name': 'segment_identifies',
    'column_name': 'anonymous_id'},
   {'database_name': 'mapbox_customer_data',
    'table_name': 'segment_identifies',
    'column_name': 'user_id'}],
  'b': [{'database_name': 'mapbox_customer_data',
    'table_name': 'accounts',
    'column_name': 'email'},
   {'database_name': 'mapbox_customer_data',
    'table_name': 'accounts',
    'column_name': 'id'},
   {'database_name': 'mapbox_customer_data',
    'table_name': 'accounts',
    'column_name': 'dt'},
   {'database_name': 'mapbox_customer_data',
    'table_name': 'accounts',
    'column_name': 'created'}],
  'c': [{'database_name': 'mapbox_customer_data',
    'table_name': 'segment_pages',
    'column_name': 'context_campaign_name'},
   {'database_name': 'mapbox_customer_data',
    'table_name': 'segment_pages',
    'c

In [58]:
query = query_dict['no alias']['e']['main']
formatter = column_parser.Parser(query)
formatted = formatter.format_query(query)
formatted

"SELECT sfdc_accounts.platform,\n       sfdc_accounts.mobile_os,\n       sfdc_accounts.service_metadata,\n       sfdc_cases.account,\n       sfdc_cases.num_requests,\n       sfdc_cases.owner,\n       sfdc_accounts.user_id\nLEFT JOIN\nWHERE sfdc_cases_oppty.dt > '2020-04-03'\n  AND sfdc_cases_oppty.dt < '2020-05-04'\nORDER BY 1\nGROUP BY 3\nLIMIT 20 AS sfdc_cases_oppty ON sfdc_cases_oppty.dt = sfdc_accounts.dt"

In [59]:
table_alias_mapping = formatter.get_table_names(formatted.split('\n'))
fields = formatter.match_queried_fields(query, df)


In [60]:
fields

[]